<a href="https://colab.research.google.com/github/karen-wang/class-scheduling/blob/main/Using_Topological_Sorting_to_Schedule_Classes_Efficiently.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Topological Sorting to Schedule Classes Efficiently
*by Karen Y. Wang on April 4th, 2023*

As a student, you probably have a list of classes that you need to take in order to graduate. Also, many of those classes likely have prerequisites. Since you are not allowed to take a class before completing its prequisites, how do you figure out a correct order of classes to take? One efficient and elegant solution uses an approach called **topological sorting**. In this tutorial, we will explain how to use topological sorting to write a class-scheduling Python function.

## Overview

### Audience

This tutorial is for university computer science students who are familiar with the following topics:
- Programming
- Data structures including graphs
- Combinatorics
- Complexity analysis

### What this document covers
- Explanation of what topological sorting is and why it's useful
- Application of topological sorting to a class-scheduling problem

### What this document doesn't cover
- Detailed implementation of topological sorting

## Input

Our input will be a Python dictionary containing classes and their prequisites, if any.

In [5]:
classes = {
    "Intro to Programming": [],
    "Computer Systems": ["Intro to Programming"],
    "Discrete Math": [],
    "Probability": ["Discrete Math"],
    "Algorithms": ["Computer Systems", "Discrete Math"],
}

## Output

We will output a list of classes that, if taken in-order, ensures all prequisites are satisfied. There might be many correct outputs—returning just one of them is enough. For the input above, a correct output is below:

```
Intro to Programming
Discrete Math
Computer Systems
Probability
Algorithms
```

## An Inefficient Approach

Let's first explore a naive solution that does not use topological sorting and is relatively inefficient. The approach is to generate all possible orderings of classes, and return the first one that fulfills all prequisites.

Here is some code that implements this solution.

In [6]:
import itertools
from typing import Collection, Dict, List, Optional, Set


def prereqs_are_satisfied(class_ordering: List[str],
                          classes: Dict[str, Collection[str]]) -> bool:
  """Returns whether or not the prereqs are satisfied for the given class ordering."""
  # Keep track of which classes we've already taken.
  classes_taken = set()
  for classname in class_ordering:
    prereqs = set(classes[classname])
    # {}.issubset({'Class A'})                     --> True
    # {'Class A'}.issubset({'Class A'})            --> True
    # {'Class A'}.issubset({'Class A', 'Class B'}) --> True
    # {'Class A'}.issubset({})                     --> False
    if prereqs.issubset(classes_taken):
      classes_taken.add(classname)
      continue
    else:
      return False
  return True

def solve_using_naive_approach(classes: Dict[str, Collection[str]]) -> List[str]:
  """Returns a correct ordering of classes."""
  # itertools.permutations('ABC') --> ABC, ACB, BAC, BCA, CAB, CBA
  for class_ordering in itertools.permutations(classes.keys()):
    if prereqs_are_satisfied(class_ordering, classes):
      # Return the first class ordering that satisfies prereqs.
      return class_ordering
  raise ValueError("No valid class ordering could be found.")

In [7]:
print(solve_using_naive_approach(classes))

('Intro to Programming', 'Computer Systems', 'Discrete Math', 'Probability', 'Algorithms')


First, let's analyze the time complexity of the `prereqs_are_satisfied` helper function. Let N represent the number of classes in class_ordering. Assuming that `issubset()` runs in constant time, this function runs in `O(N)` time.

Next, let's analyze the main function, `solve_using_naive_approach`. Since `N` represents the number of classes, there are `N!` possible permutations of classes. For each possible permutation we call the `prereqs_are_satisfied` helper function. So the time complexity of our naive solution is `O(N! * N)`.

## What is Topological Sorting?

### Topological Maps

To understand the concept behind topological sorting, let us first look at **topological maps**. In cartography, a topological map is a simplified map which only keeps information relevant to the viewer. One example is a subway map, which displays information that a subway rider would be interested in such as lines and station names. On the other hand, it removes unnecessary details such as aboveground roads.

<img src="https://drive.google.com/uc?id=1BcwLcbAo301FaNVbP8qy4ys_ku6GZX9p" height="150" alt="San Francisco subway map.">

How does this apply to our class-scheduling problem? Well, let's try drawing the relationship between classes and their prerequisites as a diagram.

<div><img src="https://drive.google.com/uc?id=1FLZZnfFB2tkeKtimh5L56DQOdEJvObS0" alt="Drawing" height=100/>
</div>

As you can see, this diagram is actually a topological map of dependencies between classes. An incoming arrow represents a dependency. For example, in order to take `Computer Systems`, we need to first take `Intro to Programming`.

### Directed Acylic Graphs (DAGs)

Computer scientists have another name for this type of map: a **directed acylic graph**, or **DAG** for short. Recall that a graph consists of a set of **nodes** and a set of **edges** between those nodes. In an undirected graph, edges are non-directional and are typically represented as lines. In a directed graph, however, edges are directional, and are typically represented as arrows (like in the diagram above).

A DAG is a special type of directed graph that does not contain any cycles. In other words, there are no closed loops within the graph. 

Why is this important? Imagine if there was a closed loop in the graph. 

<img src="https://drive.google.com/uc?id=1FLZZnfFB2tkeKtimh5L56DQOdEJvObS0" height="150" alt="Class dependency diagram containing a cycle.">

This graph is directed but contains a cycle. It is not a DAG. According to this graph, you would need to take `Class A` before `Class B`, and `Class B` before `Class C`. But, you would also need to take `Class C` before `Class A`! This is impossible. If your class list looks like this, you should probably complain to the advising department.

### Topological Ordering

We've established that the dependencies between classes can be represented as a DAG. An important property of a DAG is that it always contain at least one **topological ordering**. A topological ordering of a directed graph is a list of nodes that, if traversed in-order, ensures that all arrows are "followed" in the right direction. For example, a topological ordering of the graph below is `Intro to Programming`, `Discrete Math`, `Computer Systems`, `Probability`, and `Algorithms`.

<img src="https://drive.google.com/uc?id=1Rtx4a4yZlDJ9DLsJ0mTIH3c8iOMAIMcf" height="150" alt="Diagram showing dependencies between classes.">

Does this list look familiar to you? This list, in fact, can also be interpreted as a correct ordering of classes to take to ensure all prequisites are fulfilled. This is exactly the output that we want to return in our class-scheduling function!

### Implementation Overview

To find a topological ordering of a DAG, we need to perform a topological sort. There are a number of different ways to implement topological sorting. The Python graph library [`networkx`](https://networkx.org) uses an approach called Kahn's algorithm. The algorithm pseudocode is below.

```
L <- Empty list that will contain the sorted elements
S <- Set of all nodes with no incoming edge

while S is not empty do
    remove a node N from S
    add N to L
    for each node M with an edge E from N to M do
        remove edge E from the graph
        if M has no other incoming edges then
            insert M into S

if graph has edges then
    return error  # graph has at least one cycle
else 
    return L  # a topologically sorted order
```

At a high level, we first find the set of all nodes with no incoming edges. Recall that an incoming edge represents a dependency. Therefore, this set can also be thought of as all nodes with no dependencies. In the context of our class-scheduling problem, these are all classes with no prerequisites (`Intro to Programming` and `Discrete Math`).

We continuously add nodes to the set by following edges to connected nodes. A connected node represents a class whose prequisites we have already fulfilled, so we can add it our set. We keep on doing this until no nodes remain.

The runtime of Kahn's algorithm is efficient: `O(N + E)`, where `N` is the number of nodes and `E` is the number of edges.

### The Final Solution

To wrap-up this tutorial, let's use the `networkx` implementation of topological sort to write a class-scheduling function.

In [2]:
import networkx

from typing import Collection, Dict, List

def build_graph(classes: Dict[str, Collection[str]]) -> networkx.DiGraph:
  G = networkx.DiGraph()  # Create an empty directed graph.
  for classname, prereqs in classes.items():
    G.add_node(classname)
    for prereq in prereqs:
      G.add_edge(prereq, classname)
  return G

def solve_using_topological_sort(classes: Dict[str, Collection[str]]) -> List[str]:
  """Returns a correct ordering of classes."""
  G = build_graph(classes)
  return list(networkx.topological_sort(G))
  

In [4]:
print(solve_using_topological_sort(classes))

['Intro to Programming', 'Discrete Math', 'Computer Systems', 'Probability', 'Algorithms']


## Conclusion

In summary, topological sorting is an efficient approach to scheduling classes. This approach can also be used for many other dependency-resolution problems. Try it out for yourself!

### Further reading
For more details about the implementation of Kahn's algorithm, please refer to the official [`networkx` guide](https://networkx.org/nx-guides/content/algorithms/dag/index.html).